# 0. Context :

The following code scrape data of ASX companies from Yahoo Finance from (2020 - 2024) and export it into stock_data.csv for analysis.

### Install Necessary Libraries :

In [1]:
!pip install yfinance

In [2]:
import pandas_datareader as web
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf

from pandas.tseries.offsets import BDay

# 1. Get Data :

In [3]:
# URL of the list of ASX All ordinaries companies
file_url = 'https://raw.githubusercontent.com/RatanaSovann/Stock-Clustering-with-ML/main/asx_list.xlsx'

asx_table = pd.read_excel(file_url)
asx_tickers = asx_table['Code'].tolist()
asx_tickers = [str(ticker) + ".AX" for ticker in asx_tickers] ## Add .AX to specify ASX200 stock for Yf

In [4]:
asx_tickers

['3PL.AX',
 '4DX.AX',
 '29M.AX',
 'ABG.AX',
 'ASK.AX',
 'AX1.AX',
 'ACF.AX',
 'ADH.AX',
 'ABC.AX',
 'ADT.AX',
 'AIS.AX',
 'AFP.AX',
 'AGL.AX',
 'A1M.AX',
 'AGI.AX',
 'AIZ.AX',
 'ALK.AX',
 'AQZ.AX',
 'A4N.AX',
 'ALQ.AX',
 'ALU.AX',
 'AWC.AX',
 'AMH.AX',
 'AMC.AX',
 'AMP.AX',
 'ALD.AX',
 'ANN.AX',
 'AVR.AX',
 'ANZ.AX',
 'APA.AX',
 'APM.AX',
 'ARU.AX',
 'ARB.AX',
 'LTM.AX',
 'ARF.AX',
 'ALI.AX',
 'ARG.AX',
 'ALL.AX',
 'A1N.AX',
 'APZ.AX',
 'ASX.AX',
 'A11.AX',
 'ALX.AX',
 'ATA.AX',
 'AUB.AX',
 'AIA.AX',
 'AD8.AX',
 'AMI.AX',
 'AZJ.AX',
 'ABB.AX',
 'ASB.AX',
 'ANG.AX',
 'AAC.AX',
 'ACL.AX',
 'AEF.AX',
 'AFG.AX',
 'AFI.AX',
 'AUI.AX',
 'ASG.AX',
 'BOQ.AX',
 'BMN.AX',
 'BAP.AX',
 'BSE.AX',
 'BCI.AX',
 'BPT.AX',
 'BLX.AX',
 'BGA.AX',
 'BGL.AX',
 'BFG.AX',
 'BEN.AX',
 'BHP.AX',
 'BKI.AX',
 'SQ2.AX',
 'BSL.AX',
 'BLD.AX',
 'BOE.AX',
 'BOT.AX',
 'BOC.AX',
 'BRN.AX',
 'BXB.AX',
 'BVS.AX',
 'BRE.AX',
 'BRG.AX',
 'BKW.AX',
 'BGP.AX',
 'BFL.AX',
 'BDM.AX',
 'BWP.AX',
 'CAJ.AX',
 'CMM.AX',
 'CSC.AX',

### Test Yahoo Finance

In [5]:
# Define the ASX ticker
ticker_symbol = '29M.AX'

# Fetch the ticker data
ticker = yf.Ticker(ticker_symbol)

# Attempt to retrieve financials
try:
    financials = ticker.financials
    balance_sheet = ticker.balance_sheet
    cashflow = ticker.cashflow

    if financials.empty and balance_sheet.empty and cashflow.empty:
        print(f"No financial data available for {ticker_symbol}.")
    else:
        print("Financials:")
        print(financials)

        print("\nBalance Sheet:")
        print(balance_sheet)

        print("\nCashflow:")
        print(cashflow)

except Exception as e:
    print(f"Error fetching financial data for {ticker_symbol}: {e}")

Financials:
                                                     2023-12-31   2022-12-31  \
Tax Effect Of Unusual Items                         -16272600.0          0.0   
Tax Rate For Calcs                                          0.3     0.282515   
Normalized EBITDA                                  -177595000.0  142427000.0   
Total Unusual Items                                 -54242000.0          NaN   
Total Unusual Items Excluding Goodwill              -54242000.0          NaN   
Net Income From Continuing Operation Net Minori... -440463000.0  -47222000.0   
Reconciled Depreciation                             119709000.0  189399000.0   
Reconciled Cost Of Revenue                          424014000.0  527810000.0   
EBITDA                                             -231837000.0  142427000.0   
EBIT                                               -351546000.0  -46972000.0   
Net Interest Income                                 -29336000.0  -19172000.0   
Interest Expense            

In [7]:
ticker = yf.Ticker(ticker_symbol)

In [9]:
ticker.info

{'address1': 'Thurgauerstrasse 34',
 'city': 'Zurich',
 'zip': '8050',
 'country': 'Switzerland',
 'phone': '41 44 316 1717',
 'website': 'https://www.amcor.com',
 'industry': 'Packaging & Containers',
 'industryKey': 'packaging-containers',
 'industryDisp': 'Packaging & Containers',
 'sector': 'Consumer Cyclical',
 'sectorKey': 'consumer-cyclical',
 'sectorDisp': 'Consumer Cyclical',
 'longBusinessSummary': 'Amcor plc develops, produces, and sells packaging products in Europe, North America, Latin America, Africa, and the Asia Pacific regions. The company operates through two segments, Flexibles and Rigid Packaging. The Flexibles segment provides flexible and film packaging products in the food and beverage, medical and pharmaceutical, fresh produce, snack food, personal care, and other industries. The Rigid Packaging segment offers rigid containers for various beverage and food products, including carbonated soft drinks, water, juices, sports drinks, milk-based beverages, spirits and

### 1.2 Get all the financial metric for each stock

In [10]:
df_dict = {}

for ticker_symbol in asx_tickers:
    try:
        # Fetch the ticker data
        ticker = yf.Ticker(ticker_symbol)
        financials = ticker.financials
        balance_sheet = ticker.balance_sheet
        cashflow = ticker.cashflow

        # Fect sector
        info = ticker.info
        sector = info.get('sector')
        name = info.get('longName')


        if financials.empty and balance_sheet.empty and cashflow.empty:
            print(f"No financial data available for {ticker_symbol}.")
            continue


        # Extract necessary financial metrics directly from DataFrames
        net_income = financials.loc['Net Income'].dropna()  # Net Income
        EPS = financials.loc['Diluted EPS'].dropna()  # Earnings per share
        free_cashflow = cashflow.loc['Free Cash Flow'].dropna()  # Free cash flow
        stockholders_equity = balance_sheet.loc['Stockholders Equity'].dropna()  # Stockholders' Equity
        book_val = balance_sheet.loc['Tangible Book Value'].dropna()  # Tangible Book Value
        share_issued = balance_sheet.loc['Share Issued'].dropna()  # Number of Shares Issued
        cash = balance_sheet.loc['Cash And Cash Equivalents'].dropna()  # Cash and Cash Equivalents


        # Check if 'Current Liabilities' exists, else use 'Current Debt' (Banking companies)
        if 'Current Liabilities' in balance_sheet.index:
            current_liabilities = balance_sheet.loc['Current Liabilities'].dropna()
        elif 'Current Debt' in balance_sheet.index:
            current_liabilities = balance_sheet.loc['Current Debt'].dropna()
        else:
            current_liabilities = pd.Series(index=balance_sheet.columns)  # Create an empty Series with the same columns

        joined_df = pd.concat([net_income, EPS, free_cashflow, stockholders_equity, book_val, share_issued, current_liabilities, cash], axis=1)
        joined_df['Ticker'] = ticker_symbol
        joined_df['Sector'] = sector
        joined_df['Name'] = name

        # Append the DataFrame to the dictionary
        df_dict[ticker_symbol] = joined_df

    except Exception as e:
        print(f"Error processing {ticker_symbol}: {e}")

# Combine all DataFrames into one final DataFrame
final_df = pd.concat(df_dict.values())

No financial data available for KKC.AX.
No financial data available for MGF.AX.
No financial data available for PCI.AX.
No financial data available for RG8.AX.
No financial data available for RF1.AX.
No financial data available for VG1.AX.
No financial data available for WQG.AX.


In [11]:
final_df

,Net Income,Diluted EPS,Free Cash Flow,Stockholders Equity,Tangible Book Value,Share Issued,Current Liabilities,Cash And Cash Equivalents,Ticker,Sector,Name,0,Current Debt
2023-06-30,6370000.0,0.023,4642000.0,200691000.0,-5122000.0,276484170.0,57757000.0,14010000.0,3PL.AX,Consumer Defensive,3P Learning Limited,NaN,NaN
2022-06-30,-536000.0,-0.0019,7651000.0,193901000.0,-7046000.0,276484170.0,59518000.0,31127000.0,3PL.AX,Consumer Defensive,3P Learning Limited,NaN,NaN
2021-06-30,-9369000.0,-0.0615,-4558000.0,194832000.0,-12821000.0,276484170.0,55493000.0,24906000.0,3PL.AX,Consumer Defensive,3P Learning Limited,NaN,NaN
2020-06-30,1550000.0,0.0111,2417000.0,26267000.0,5402000.0,139484170.0,35612000.0,27083000.0,3PL.AX,Consumer Defensive,3P Learning Limited,NaN,NaN
2023-06-30,-31459798.0,-0.1,-23384007.0,71460377.0,66377721.0,345132572.0,17071692.0,69576373.0,4DX.AX,Healthcare,4DMedical Limited,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-30,261839000.0,2.43,154140000.0,1271999000.0,1271999000.0,107637649.0,120999000.0,135817000.0,ZIM.AX,Basic Materials,Zimplats Holdings Limited,NaN,NaN
2023-06-30,-377015000.0,-0.5233,-249595000.0,234340000.0,-116533000.0,824647000.0,222445000.0,275910000.0,ZIP.AX,Financial Services,Zip Co Limited,NaN,NaN
2022-06-30,-1105093000.0,-1.8126,-781932000.0,436734000.0,21640000.0,687936000.0,156362000.0,299695000.0,ZIP.AX,Financial Services,Zip Co Limited,NaN,NaN
2021-06-30,-658774000.0,-1.2705,30831000.0,1168376000.0,171709000.0,562136000.0,73453000.0,330201000.0,ZIP.AX,Financial Services,Zip Co Limited,NaN,NaN


In [12]:
# Merge Current debt as Current liabilities

final_df['Current Debt'] = final_df['Current Debt'].fillna(0)
final_df['Current Liabilities'] = final_df['Current Liabilities'].fillna(0)
final_df['Current Liabilities'] = final_df['Current Liabilities'] + final_df['Current Debt']

final_df = final_df.drop(columns=['Current Debt', 0])

In [13]:
df = final_df.copy()

In [14]:
df['Sector']

2023-06-30    Consumer Defensive
2022-06-30    Consumer Defensive
2021-06-30    Consumer Defensive
2020-06-30    Consumer Defensive
2023-06-30            Healthcare
                     ...        
2020-06-30       Basic Materials
2023-06-30    Financial Services
2022-06-30    Financial Services
2021-06-30    Financial Services
2020-06-30    Financial Services
Name: Sector, Length: 1960, dtype: object

In [15]:
# Check missing Sector
mask_df = df[df['Sector'].isna()]
mask_df

,Net Income,Diluted EPS,Free Cash Flow,Stockholders Equity,Tangible Book Value,Share Issued,Current Liabilities,Cash And Cash Equivalents,Ticker,Sector,Name
2021-06-30,28904000.0,0.039,53140000.0,175991000.0,102429000.0,818591000.0,69594000.0,16019000.0,ASK.AX,None,Abacus Storage King
2022-06-30,42072000.0,NaN,83168000.0,240510000.0,167557000.0,892429000.0,113163000.0,38033000.0,ASK.AX,None,Abacus Storage King
2022-06-30,141073000.0,0.2316,385700000.0,1685426000.0,1685426000.0,611162000.0,0.0,1310828000.0,LSF.AX,None,L1 Long Short Fund Limited
2021-06-30,514192000.0,0.8272,-503272000.0,1574311000.0,1574311000.0,606619000.0,0.0,483995000.0,LSF.AX,None,L1 Long Short Fund Limited
2020-06-30,-21650000.0,-0.0327,-557676000.0,1140530000.0,1140530000.0,649064000.0,0.0,559312000.0,LSF.AX,None,L1 Long Short Fund Limited




*   Abacus Storage King is part of Abacus Group. Since Abacus Property Group stock already exists we remove this ASK.AX
*   LSF is in the Financial Sector





In [17]:
# Remove ASK.AX
df = df[df['Ticker'] != 'ASK.AX']
# Manually put LSF.AX sector as Financials
df.loc[df['Ticker'] == 'LSF.AX', 'Sector'] = 'Financials'


In [20]:
df['Sector'].value_counts()

Sector
Basic Materials           410
Financial Services        347
Industrials               215
Consumer Cyclical         201
Real Estate               163
Healthcare                161
Technology                149
Energy                     93
Communication Services     87
Consumer Defensive         78
Utilities                  35
Financial                  16
Financials                  3
Name: count, dtype: int64

In [21]:
# Combine Financial, Financials and Financial Services and name it Financials
df.loc[df['Sector'].isin(['Financial', 'Financials', 'Financial Services']), 'Sector'] = 'Financials'

In [24]:
df['Sector'].value_counts()

Sector
Basic Materials           410
Financials                366
Industrials               215
Consumer Staples          201
Real Estate               163
Healthcare                161
Technology                149
Energy                     93
Communication Services     87
Consumer Discretionary     78
Utilities                  35
Name: count, dtype: int64

In [23]:
# Rename Consumer Cyclical to Consumer Staples
df.loc[df['Sector'] == 'Consumer Cyclical', 'Sector'] = 'Consumer Staples'
# Rename Consumer Defensive to Consumer Discretionary
df.loc[df['Sector'] == 'Consumer Defensive', 'Sector'] = 'Consumer Discretionary'

The sectors follows The Australian Stock Exchange (ASX) useage of the Global Industry Classification Standard (GICS) method of categorising companies

### 1.3 Get Historical Data for Stock :

In [25]:
# Function to adjust weekends to the next business day
def roll_forward_to_next_business_day(date):
    # Check if the date is a weekend
    if date.weekday() >= 5:  # Saturday or Sunday
        date += BDay(1)  # Add one business day

    # Check if the adjusted date is New Year's Day (Jan 1)
    if date.month == 1 and date.day == 1:
        date += BDay(1)  # Add one business day

    return date

In [26]:
# Initialize a list to store DataFrames for each ticker
df_list = []

for index, row in final_df.iterrows():
  try:
    date = index
    ticker_symbol = row['Ticker']

    # Download historical data
    data = yf.download(ticker_symbol, start="2020-01-01", end="2024-06-04")

    if data.empty:
      print(f"No data available for {ticker_symbol} on {date}")
      continue

    prices = []
    adjusted_date = roll_forward_to_next_business_day(date)

    if adjusted_date in data.index:
      prices.append((adjusted_date, data.loc[adjusted_date, 'Close']))

      # Calculate 13 weeks (approximately 65 trading days) price change and volatility
      end_date = adjusted_date
      start_date = end_date - pd.Timedelta(weeks=13)
      past_prices = data.loc[start_date:end_date, 'Close']

      price_change = ((past_prices[-1] - past_prices[0]) / past_prices[0]) * 100
      volatility = past_prices.std()

      # Convert to DataFrame for better readability
      prices_df = pd.DataFrame(prices, columns=['Date', 'Close'])
      prices_df['Ticker'] = ticker_symbol  # Add ticker symbol column
      prices_df['price_change'] = price_change
      prices_df['volatility'] = volatility

      # Add the DataFrame to the list
      df_list.append(prices_df)

    else:
      print(f"No data available for {ticker_symbol} on {adjusted_date}")

  except:
    print(f"Error processing {ticker_symbol} on {date}")

combined_df = pd.concat(df_list)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


No data available for 29M.AX on 2020-12-31 00:00:00
No data available for 29M.AX on 2023-01-02 00:00:00


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


No data available for ABG.AX on 2023-06-30 00:00:00
No data available for ABG.AX on 2022-06-30 00:00:00
No data available for ABG.AX on 2021-06-30 00:00:00
No data available for ABG.AX on 2020-06-30 00:00:00


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


No data available for XRO.AX on 2024-04-01 00:00:00


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


No data available for YAL.AX on 2019-12-31 00:00:00
No data available for YAL.AX on 2023-01-02 00:00:00


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [27]:
past_prices


Date
2020-03-31    1.585
2020-04-01    1.650
2020-04-02    1.600
2020-04-03    1.705
2020-04-06    1.970
              ...  
2020-06-24    5.900
2020-06-25    5.490
2020-06-26    5.450
2020-06-29    5.090
2020-06-30    5.230
Name: Close, Length: 63, dtype: float64

In [28]:
combined_df

,Date,Close,Ticker,price_change,volatility
0,2023-06-30,1.100,3PL.AX,-11.999998,0.047663
0,2022-06-30,1.235,3PL.AX,-20.322577,0.123008
0,2021-06-30,1.310,3PL.AX,1.550386,0.029412
0,2020-06-30,0.860,3PL.AX,2.380957,0.055610
0,2023-06-30,0.670,4DX.AX,109.375010,0.173467
...,...,...,...,...,...
0,2020-06-30,10.390,ZIM.AX,17.004507,0.640418
0,2023-06-30,0.410,ZIP.AX,-26.785715,0.044526
0,2022-06-30,0.440,ZIP.AX,-70.370371,0.318077
0,2021-06-30,7.570,ZIP.AX,2.574526,0.784955


### Merge Financial and Price Data together

In [29]:
# Merge column with price
df = df.merge(combined_df,how = 'right', on='Ticker') # Merge on price data ()

In [30]:
df['Sector'].isna().sum()

0

In [31]:
# Reorder the columns
columns_order = ['Date', 'Ticker', 'Name', 'Sector', 'Close', 'price_change', 'volatility', 'Net Income', 'Diluted EPS', 'Free Cash Flow', 'Stockholders Equity', 'Tangible Book Value', 'Share Issued', 'Current Liabilities', 'Cash And Cash Equivalents']
df = df.reindex(columns=columns_order)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7072 entries, 0 to 7071
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Date                       7072 non-null   datetime64[ns]
 1   Ticker                     7072 non-null   object        
 2   Name                       7070 non-null   object        
 3   Sector                     7072 non-null   object        
 4   Close                      7072 non-null   float64       
 5   price_change               7072 non-null   float64       
 6   volatility                 7072 non-null   float64       
 7   Net Income                 6845 non-null   object        
 8   Diluted EPS                6829 non-null   object        
 9   Free Cash Flow             6857 non-null   object        
 10  Stockholders Equity        6912 non-null   object        
 11  Tangible Book Value        6912 non-null   object        
 12  Share 

In [33]:
# Convert the specified columns to float
columns_to_convert = [
    'Net Income', 'Diluted EPS', 'Free Cash Flow',
    'Stockholders Equity', 'Tangible Book Value',
    'Share Issued', 'Cash And Cash Equivalents'
]

for column in columns_to_convert:
    df[column] = pd.to_numeric(df[column], errors='coerce')

# Display the dtypes to confirm the conversion
print(df.dtypes)

Date                         datetime64[ns]
Ticker                               object
Name                                 object
Sector                               object
Close                               float64
price_change                        float64
volatility                          float64
Net Income                          float64
Diluted EPS                         float64
Free Cash Flow                      float64
Stockholders Equity                 float64
Tangible Book Value                 float64
Share Issued                        float64
Current Liabilities                 float64
Cash And Cash Equivalents           float64
dtype: object


In [34]:
df

,Date,Ticker,Name,Sector,Close,price_change,volatility,Net Income,Diluted EPS,Free Cash Flow,Stockholders Equity,Tangible Book Value,Share Issued,Current Liabilities,Cash And Cash Equivalents
0,2023-06-30,3PL.AX,3P Learning Limited,Consumer Discretionary,1.100,-11.999998,0.047663,6.370000e+06,0.0230,4642000.0,2.006910e+08,-5122000.0,276484170.0,57757000.0,14010000.0
1,2023-06-30,3PL.AX,3P Learning Limited,Consumer Discretionary,1.100,-11.999998,0.047663,-5.360000e+05,-0.0019,7651000.0,1.939010e+08,-7046000.0,276484170.0,59518000.0,31127000.0
2,2023-06-30,3PL.AX,3P Learning Limited,Consumer Discretionary,1.100,-11.999998,0.047663,-9.369000e+06,-0.0615,-4558000.0,1.948320e+08,-12821000.0,276484170.0,55493000.0,24906000.0
3,2023-06-30,3PL.AX,3P Learning Limited,Consumer Discretionary,1.100,-11.999998,0.047663,1.550000e+06,0.0111,2417000.0,2.626700e+07,5402000.0,139484170.0,35612000.0,27083000.0
4,2022-06-30,3PL.AX,3P Learning Limited,Consumer Discretionary,1.235,-20.322577,0.123008,6.370000e+06,0.0230,4642000.0,2.006910e+08,-5122000.0,276484170.0,57757000.0,14010000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7067,2021-06-30,ZIP.AX,Zip Co Limited,Financials,7.570,2.574526,0.784955,-1.994100e+07,-0.0531,-4849000.0,2.054000e+08,126866000.0,390390000.0,22286000.0,32712000.0
7068,2020-06-30,ZIP.AX,Zip Co Limited,Financials,5.230,229.968448,1.662662,-3.770150e+08,-0.5233,-249595000.0,2.343400e+08,-116533000.0,824647000.0,222445000.0,275910000.0
7069,2020-06-30,ZIP.AX,Zip Co Limited,Financials,5.230,229.968448,1.662662,-1.105093e+09,-1.8126,-781932000.0,4.367340e+08,21640000.0,687936000.0,156362000.0,299695000.0
7070,2020-06-30,ZIP.AX,Zip Co Limited,Financials,5.230,229.968448,1.662662,-6.587740e+08,-1.2705,30831000.0,1.168376e+09,171709000.0,562136000.0,73453000.0,330201000.0


In [35]:
df.isna().sum()

Date                           0
Ticker                         0
Name                           2
Sector                         0
Close                          0
price_change                   0
volatility                     0
Net Income                   227
Diluted EPS                  243
Free Cash Flow               215
Stockholders Equity          160
Tangible Book Value          160
Share Issued                 160
Current Liabilities            0
Cash And Cash Equivalents    176
dtype: int64

### The mismatch in reporting periods between stocks is what generating most of these N/A

In [36]:
df['Ticker'].nunique()

484

In [37]:
df_2 = df.dropna()

In [38]:
df_2['Ticker'].nunique()

483

In [39]:
df_2.to_csv('stock_data.csv', index=False)